In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

# data input
- mis fuentes:
  
https://www.kaggle.com/datasets/fcpercival/160k-spotify-songs-sorted?resource=download&select=data.csv

https://www.kaggle.com/datasets/asifsadmine/spotify-playlists-dataset

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [2]:
df = pd.read_csv('datos/data.csv')
df.head()

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit
0,0gNNToCW3qjabgTyBSjt3H,!Que Vida! - Mono Version,['Love'],220560,11/1/66,1966,0.5250,0.600,0.540,0.00305,0.100,-11.803,0.0328,125.898,0.547,1,9,26,0
1,0tMgFpOrXZR6irEOLNWwJL,"""40""",['U2'],157840,2/28/83,1983,0.2280,0.368,0.480,0.70700,0.159,-11.605,0.0306,150.166,0.338,1,8,21,0
2,2ZywW3VyVx6rrlrX75n3JB,"""40"" - Live",['U2'],226200,8/20/83,1983,0.0998,0.272,0.684,0.01450,0.946,-9.728,0.0505,143.079,0.279,1,8,41,0
3,6DdWA7D1o5TU2kXWyCLcch,"""40"" - Remastered 2008",['U2'],157667,2/28/83,1983,0.1850,0.371,0.545,0.58200,0.183,-9.315,0.0307,150.316,0.310,1,8,37,0
4,3vMmwsAiLDCfyc1jl76lQE,"""40"" - Remastered 2008",['U2'],157667,2/28/83,1983,0.1850,0.371,0.545,0.58200,0.183,-9.315,0.0307,150.316,0.310,1,8,35,0


# análisis exploratorio
mi dataset tiene los siguientes campos:
- `id` -- tipo `object`. id único de cada canción para Spotify.
- `name` -- tipo `object`. nombre de la canción 
- `artists` -- tipo `object`. nombre del intérprete, puede tener caracteres de tipo cilírico
- `duration_ms` -- tipo `np.int64`. campo numérico, indica la longitud de la canción en ms
- `release_date` -- tipo `object`. fecha de publicación del tema. se va a tener que recastear y limpiar para poder homogeneizarlo.
- `acousticness` -- tipo `np.float64`. puntúa del 0 al 1 si la canción es acústica. Cuanto mayor sea, más bailable es el tema.
- `danceability` -- tipo `np.float64`. puntúa del 0 al 1 si la canción es bailable, basándose en criterios de tempo, ritmo estable, potencia del beat y regularidad. Cuanto mayor el valor, más bailable es el tema.
- `energy` -- tipo `np.float64`. puntúa del 0 al 1 si la canción es "energética"; se evalua la intensidad y la actividad de la canción, teniendo en cuenta factores como la intensidad y la rapidez del tempo del tema.
- `instrumentalness` -- tipo `np.float64`. puntúa del 0 al 1 si se trata de una canción con o sin cantantes. Valores por encima de 0.5 consideran que la pisa es instrumental.
- `liveness` -- tipo `np.float64`. puntúa del 0 al 1 si la cancion se trata de una grabación en directo o de estudio. En torno a 0.8 indica que se trata de una pista grabada en directo.
- `loudness` -- tipo `np.float64`. indica en decibelios (db) la intensidad sonora de la pista, puntuando la "potencia" del sonido. El valor de los decibelios es la media de la pista, pudiendo variar entre -60  0.
- `speechiness` -- tipo `np.float64`. puntúa del 0 al 1 la presencia de palabras recitadas en una pista -- según la cantidad de texto hablado que aparece en la pista. A partir del umbral del 0.66 se considera que la pista está compuesta mayormente de palabras recitadas. Si los valores varían entre 0.33 y 0.66 se puede considerar que contiene tanto música como palabras, en secciones o superpuestas, como pueda ser el caso de una canción de rap. Valores por debajo de 0.33 indican que la canción es mayormente instrumental y no-recitada.
- `tempo` -- tipo `np.float64`. estimación del tempo de una pista en beats per minute (BPM). En terminología musical, el tempo indica la velocidad de la canción. Puede variar entre 0 a 245BPM.
- `valence` -- tipo `np.float64`. valora del 0 al 1 la "positividad" que tiene la pista. Canciones con una `valence` alta suenan más positivas (ej. felices, eufóricas, alegres), que canciones con un valor de `valence` inferior que se perciben más negativas (ej. triste, depresivas, enfadada)
- `mode` -- tipo `np.int64`. indica el modo de la canción (mayor = 1, menor = 0)
- `key` -- tipo `np.int64`. tono en el que está la canción, puntuándolo de 0 a 11, siguiendo la notación indicada aquí: https://en.wikipedia.org/wiki/Pitch_class (Ej. 0 = C, 1 = C#//D♭). Si no se detecta el tono, se considerará el valor -1.
- `popularity`  -- tipo `np.int64`. popularidad de la canción
- `explicit` -- tipo `np.int64`. indica si la canción tiene o no letras explícitas.

## primero voy a mirar los tipos de datos que tiene cada columna:

In [3]:
df.dtypes

id                   object
name                 object
artists              object
duration_ms           int64
release_date         object
year                  int64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
mode                  int64
key                   int64
popularity            int64
explicit              int64
dtype: object

todos los campos están bien casteados menos el de fecha, que será corregido a continuación

## vamos a ver si el dataframe tuviera algún nulo con el método `info()`

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169907 entries, 0 to 169906
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                169907 non-null  object 
 1   name              169907 non-null  object 
 2   artists           169907 non-null  object 
 3   duration_ms       169907 non-null  int64  
 4   release_date      169907 non-null  object 
 5   year              169907 non-null  int64  
 6   acousticness      169907 non-null  float64
 7   danceability      169907 non-null  float64
 8   energy            169907 non-null  float64
 9   instrumentalness  169907 non-null  float64
 10  liveness          169907 non-null  float64
 11  loudness          169907 non-null  float64
 12  speechiness       169907 non-null  float64
 13  tempo             169907 non-null  float64
 14  valence           169907 non-null  float64
 15  mode              169907 non-null  int64  
 16  key               16

no parece que ninguna de las columnas tenga duplicados.

## check de los valores duplicados de df con `duplicated()`

In [56]:
df.duplicated(subset = df.columns).sum()

0

## vamos a mirar las estadísticas características de cada una de las columnas mediante `describe()`

In [5]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,169907,169907,0gNNToCW3qjabgTyBSjt3H,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,169907,132938,Summertime,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
artists,169907,33375,['Эрнест Хемингуэй'],1215,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration_ms,169907.0,NaN,NaN,NaN,231407.085988,121322.336392,5108.0,171040.0,208600.0,262966.5,5403500.0
release_date,169907,10882,1945,1449,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,169907.0,NaN,NaN,NaN,1977.223234,25.593318,1921.0,1957.0,1978.0,1999.0,2020.0
acousticness,169907.0,NaN,NaN,NaN,0.493217,0.376628,0.0,0.0945,0.492,0.888,0.996
danceability,169907.0,NaN,NaN,NaN,0.538147,0.175345,0.0,0.417,0.548,0.667,0.988
energy,169907.0,NaN,NaN,NaN,0.488591,0.267391,0.0,0.263,0.481,0.71,1.0
instrumentalness,169907.0,NaN,NaN,NaN,0.161939,0.30933,0.0,0.0,0.000204,0.0868,1.0


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration_ms,169907.0,231407.085988,121322.336392,5108.0,171040.0000,208600.000000,262966.5000,5403500.000
year,169907.0,1977.223234,25.593318,1921.0,1957.0000,1978.000000,1999.0000,2020.000
acousticness,169907.0,0.493217,0.376628,0.0,0.0945,0.492000,0.8880,0.996
danceability,169907.0,0.538147,0.175345,0.0,0.4170,0.548000,0.6670,0.988
energy,169907.0,0.488591,0.267391,0.0,0.2630,0.481000,0.7100,1.000
instrumentalness,169907.0,0.161939,0.309330,0.0,0.0000,0.000204,0.0868,1.000
liveness,169907.0,0.206692,0.176797,0.0,0.0984,0.135000,0.2630,1.000
loudness,169907.0,-11.370311,5.666795,-60.0,-14.4700,-10.474000,-7.1180,3.855
speechiness,169907.0,0.094058,0.149938,0.0,0.0349,0.045000,0.0754,0.969
tempo,169907.0,116.947850,30.727079,0.0,93.5160,114.777000,135.7120,244.091


In [7]:
df.describe(include='object').T

,count,unique,top,freq
id,169907,169907,0gNNToCW3qjabgTyBSjt3H,1
name,169907,132938,Summertime,62
artists,169907,33375,['Эрнест Хемингуэй'],1215
release_date,169907,10882,1945,1449


# cast a las fechas
ya se observa que el campo `release_date` se puede castear como fecha, vamos a ver si todos los campos tienen la misma estructura:

In [8]:
df['release_date'].value_counts()

1945       1449
1949       1254
1935       1123
1948       1052
1/1/30     1047
           ... 
4/16/01       1
2/22/03       1
3/30/15       1
2/21/89       1
4/19/04       1
Name: release_date, Length: 10882, dtype: int64

hay años que no tienen día -> habrá que suponer la fecha, para poder generar un nuevo campo `date` que tenga el formato adecuado. podemos intentar ver posibilidades calculando la longitd de los datos que se almacenan en el campo `release_date`.

In [9]:
df['release_date_length'] = df['release_date'].apply(len)

In [10]:
df['release_date_length'].value_counts()

6    52683
4    50382
7    50212
8    16630
Name: release_date_length, dtype: int64

In [11]:
df_fechas = df['release_date_length'].value_counts().reset_index()
df_fechas['ratio'] = df_fechas['release_date_length'] / df.shape[0]
df_fechas

,index,release_date_length,ratio
0,6,52683,0.310070
1,4,50382,0.296527
2,7,50212,0.295526
3,8,16630,0.097877


In [12]:
df[df['release_date_length'] == 4].head(1)

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length
6,3mdPsKBeXxC4N6oKKZeBNy,"""8"" Teen",['? & The Mysterians'],166693,1966,1966,0.111,0.657,0.872,0.713,0.0857,-7.279,0.0512,136.512,0.738,1,2,17,0,4


In [13]:
df[df['release_date_length'] == 6].sample(5)

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length
72852,3wGQULikds23nGCvIJf5G4,Just the Way You Are,['The Piano Guys'],262240,5/3/13,2013,0.992,0.383,0.0678,0.9170,0.0811,-18.621,0.0417,90.578,0.1930,1,4,57,0,6
13340,1TomqIgyiZBBpU4SYfm4PF,"Bartók: Music for Strings, Percussion and Cele...","['Béla Bartók', 'Herbert von Karajan', 'Philha...",486080,7/1/51,1951,0.875,0.103,0.0448,0.8590,0.1360,-22.111,0.0394,82.655,0.0321,0,5,2,0,6
68725,3FEEfb772gpjK2J7GUsEb8,Is It Because I'm Black?,['Ken Boothe'],222667,1/1/87,1987,0.141,0.736,0.4420,0.0308,0.1020,-13.887,0.0569,147.054,0.9450,0,10,43,0,6
169022,5FVRaL3CKfTqWpS4Qqhwl1,南屏晚鐘,['崔萍'],190133,2/1/45,1945,0.861,0.486,0.1420,0.0000,0.1070,-12.160,0.0325,108.193,0.5490,1,0,7,0,6
139579,13XMn3vVhwm12NOCpkB380,The Harder They Come,['Jimmy Cliff'],220533,1/1/99,1999,0.700,0.668,0.6500,0.0000,0.1250,-6.292,0.0583,103.512,0.8480,1,1,55,0,6


In [14]:
df[df['release_date_length'] == 7].sample(5)

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length
16649,66Avcq6c51vcMGXLzHH5d7,Bloodfeather,['Highly Suspect'],234093,7/17/15,2015,0.000383,0.547,0.792,0.000259,0.106,-5.645,0.0350,97.998,0.395,0,9,55,1,7
19060,6X8qok8S6wJKd60UyJqCKp,Brindis - Remastered,['Orquesta Gran Casino'],191298,1/12/42,1942,0.754000,0.740,0.620,0.005600,0.214,-9.074,0.0437,116.051,0.936,1,10,0,0,7
161601,1VZo1zddq6LpxwMtvHPVgM,Wrecking Ball,['Mother Mother'],194320,9/23/08,2008,0.194000,0.664,0.797,0.000000,0.337,-4.473,0.0362,97.004,0.284,1,7,53,0,7
34314,47uDHNJpq4zUfOM0RFdEbV,Do Me!,['Bell Biv DeVoe'],272427,3/20/90,1990,0.017900,0.714,0.735,0.000016,0.238,-8.030,0.0373,107.822,0.704,1,7,46,0,7
120093,5D8SzvASRr7V7ZrrXsh6SY,Serpentskirt,['Cocteau Twins'],237600,1996-03,1996,0.595000,0.451,0.583,0.000030,0.195,-6.481,0.0244,77.945,0.582,0,4,35,0,7


In [15]:
df[df['release_date_length'] == 8].sample(5)

,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length
54778,5MuNxNox3zTanAFIO5KcTl,He's the Greatest Dancer,['Sister Sledge'],375400,10/30/07,2007,0.00115,0.700,0.816,0.001230,0.0901,-9.711,0.0440,113.244,0.838,0,7,56,0,8
13306,3DRH5iVqchXWQMZRPts3cr,Barrio Pobre,['Cornelio Reyna'],183278,12/19/91,1991,0.58500,0.479,0.260,0.000000,0.0580,-8.925,0.0640,170.063,0.676,1,9,34,0,8
25883,3jYW03V9XeM2w7LJ0EXQDC,Cockiness (Love It),['Rihanna'],178027,11/19/11,2011,0.00118,0.673,0.773,0.000569,0.1480,-7.099,0.0576,124.999,0.265,1,8,48,0,8
93661,71sWzf54Q0EHF2SsMjjkI4,Muñeca Cruel,['Enrique Iglesias'],261000,11/21/95,1995,0.04780,0.721,0.812,0.000268,0.1590,-7.601,0.0263,112.097,0.951,1,9,48,0,8
95653,2vUZzKx2klI6KZfuhxDEhE,Nazdik Na Aana Na Mujhe Hath Lagana,"['Geeta Dutt', 'S. D. Batish']",196325,12/31/52,1952,0.98600,0.714,0.431,0.007720,0.0907,-7.532,0.1110,80.904,0.886,0,7,0,0,8


según la longitud:
- `4`: solo tenemos información del año.
- `6`: tenemos información de año, separadores y mes y día. en este caso, los meses y días tienen solo una cifra.
- `7`: tenemos información de año, mes y día. o el mes o el día tienen 2 cifras.
- `8`: tenemos información de año, mes y día. todos tienen 2 cifras.

además hay que comprobar el tipo de delimitador que se usa para la fecha

In [16]:
print((1 - (df.shape[0] - df['release_date'].str.contains('-').sum())/df.shape[0])*100, 
    (1 - (df.shape[0] - df['release_date'].str.contains('/').sum())/df.shape[0])*100)

0.9705309375128701 69.3767767072575


se va a actuar de formas distintas según la longitud del campo `release_date`

In [17]:
df['month'] = ''
df['day'] = ''

#### largo del campo es 6
como ya hemos dicho, el largo del campo es 4 -> el campo es directamente el año. forzaremos a que el primer día del mes sea el día de pulicación del disco.

In [18]:
df_06 = df[df['release_date_length'] == 6]
df_06['release_date'].value_counts()

1/1/30    1047
1/1/40    1008
1/1/50     935
1/1/55     770
1/1/61     646
          ... 
1/4/88       1
3/8/60       1
6/3/82       1
2/9/95       1
7/9/17       1
Name: release_date, Length: 2573, dtype: int64

In [19]:
### comprobar cual es el separador de los datos de fecha
print((1 - (df_06.shape[0] - df_06['release_date'].str.contains('-').sum())/df_06.shape[0])*100, 
    (1 - (df_06.shape[0] - df_06['release_date'].str.contains('/').sum())/df_06.shape[0])*100)

0.0 100.0


In [20]:
df_06[['month', 'day', 'decade']] = df_06['release_date'].str.split('/', expand=True)
df_06[['month', 'day', 'decade']].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\67254662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_06[['month', 'day', 'decade']] = df_06['release_date'].str.split('/', expand=True)
C:\Users\annas\AppData\Local\Temp\ipykernel_23844\67254662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_06[['month', 'day', 'decade']] = df_06['release_date'].str.split('/', expand=True)


month  day  decade
1      1    30        1047
            40        1008
            50         935
            55         770
            61         646
                      ... 
7      6    95           1
3      4    10           1
            19           1
7      7    16           1
6      8    68           1
Length: 2573, dtype: int64

In [21]:
df_06['day'].unique()

array(['1', '2', '6', '7', '5', '3', '9', '8', '4'], dtype=object)

In [22]:
df_06['month'].unique()

array(['1', '3', '8', '2', '9', '7', '4', '6', '5'], dtype=object)

In [23]:
df_06['decade'].unique()

array(['12', '61', '52', '64', '53', '28', '77', '73', '07', '94', '00',
       '14', '13', '78', '08', '15', '69', '87', '93', '79', '81', '90',
       '57', '59', '58', '56', '95', '51', '97', '88', '55', '54', '01',
       '85', '09', '80', '83', '11', '76', '63', '72', '66', '60', '86',
       '68', '70', '62', '03', '84', '91', '20', '96', '99', '75', '46',
       '82', '05', '65', '67', '71', '04', '74', '92', '89', '10', '16',
       '06', '44', '18', '17', '19', '02', '40', '49', '98', '48', '50',
       '29', '30', '45', '39', '27', '26', '33', '41', '36', '24', '34',
       '32', '31', '35', '42', '47', '43', '25', '38', '37', '23'],
      dtype=object)

In [24]:
df_06['date'] = df_06['day'] + '/' + df_06['month'] + '/' + df_06['year'].astype(str)
df_06['date'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\2783625972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_06['date'] = df_06['day'] + '/' + df_06['month'] + '/' + df_06['year'].astype(str)


1/1/1930    1047
1/1/1940    1008
1/1/1950     935
1/1/1955     770
1/1/1961     646
            ... 
4/1/1988       1
8/3/1960       1
3/6/1982       1
9/2/1995       1
9/7/2017       1
Name: date, Length: 2573, dtype: int64

### largo del campo es 7
como ya hemos dicho, el largo del campo es 7 -> el campo contiene todos los elementos de fecha. forzaremos a que el primer día del mes sea el día de pulicación del disco.

In [25]:
df_07 = df[df['release_date_length'] == 7]
df_07['release_date'].value_counts()

12/1/52    148
6/12/20    145
12/1/47    134
12/1/48    110
12/1/49    103
          ... 
1/31/07      1
9/14/07      1
1/16/41      1
10/7/13      1
4/19/04      1
Name: release_date, Length: 6426, dtype: int64

In [26]:
### comprobar cual es el separador de los datos de fecha
print((1 - (df_07.shape[0] - df_07['release_date'].str.contains('-').sum())/df_07.shape[0])*100, 
    (1 - (df_07.shape[0] - df_07['release_date'].str.contains('/').sum())/df_07.shape[0])*100)

3.284075519796059 96.71592448020394


#### separador `-`
aparece el caracter `-` como separador, por lo que tendremos que separar el dataframe

In [27]:
df_07_guion = df_07[df_07['release_date'].str.contains('-')]
print(df_07_guion.shape)
df_07_guion.head(1)

(1649, 22)


,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length,month,day
140,1cHxcl608r9sfaHaishns3,'Deed I Do,['Ella Fitzgerald'],162667,1963-09,1963,0.802,0.525,0.26,0.0,0.202,-12.79,0.042,117.466,0.446,0,0,25,0,7,,


In [28]:
df_07_guion[['decade', 'month']] = df_07_guion['release_date'].str.split('-', expand=True)
df_07_guion[['decade', 'month']] .value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\3575440809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_07_guion[['decade', 'month']] = df_07_guion['release_date'].str.split('-', expand=True)


decade  month
1970    02       54
1966    03       51
1957    03       34
1963    08       33
1957    09       30
                 ..
1977    10        1
1974    10        1
1973    02        1
1970    11        1
2003    01        1
Length: 187, dtype: int64

In [29]:
df_07_guion['month'].unique()

array(['09', '08', '07', '12', '03', '10', '06', '11', '04', '02', '01',
       '05'], dtype=object)

In [30]:
df_07_guion['decade'].unique()

array(['1963', '1969', '1967', '1971', '1977', '1992', '1979', '1964',
       '1975', '1960', '1959', '1966', '1962', '1981', '1965', '1984',
       '1987', '1968', '1974', '1985', '1957', '1970', '1978', '1988',
       '1973', '1972', '1961', '1956', '1986', '1958', '1980', '1982',
       '1976', '1994', '1995', '1990', '1983', '1991', '1989', '2003',
       '1997', '1996'], dtype=object)

In [31]:
df_07_guion['date'] = '01/' + df_07_guion['month'] + '/' + df_07_guion['year'].astype(str)
df_07_guion['date'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\1997061340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_07_guion['date'] = '01/' + df_07_guion['month'] + '/' + df_07_guion['year'].astype(str)


01/02/1970    54
01/03/1966    51
01/03/1957    34
01/08/1963    33
01/09/1957    30
              ..
01/11/1987     1
01/09/1971     1
01/10/1977     1
01/01/1979     1
01/04/1987     1
Name: date, Length: 187, dtype: int64

#### separador `/`
ahora trataré los datos que tienene  `/` como separador

In [32]:
df_07_barra = df_07[df_07['release_date'].str.contains('/')]
print(df_07_barra.shape)
df_07_barra.head(1)

(48563, 22)


,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length,month,day
0,0gNNToCW3qjabgTyBSjt3H,!Que Vida! - Mono Version,['Love'],220560,11/1/66,1966,0.525,0.6,0.54,0.00305,0.1,-11.803,0.0328,125.898,0.547,1,9,26,0,7,,


In [33]:
df_07_barra[['month', 'day', 'decade']] = df_07_barra['release_date'].str.split('/', expand=True)
df_07_barra[['month', 'day', 'decade']] .value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\949077870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_07_barra[['month', 'day', 'decade']] = df_07_barra['release_date'].str.split('/', expand=True)
C:\Users\annas\AppData\Local\Temp\ipykernel_23844\949077870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_07_barra[['month', 'day', 'decade']] = df_07_barra['release_date'].str.split('/', expand=True)


month  day  decade
12     1    52        148
6      12   20        145
12     1    47        134
            48        110
6      16   66        103
                     ... 
7      14   38          1
2      28   03          1
7      14   10          1
2      28   05          1
4      27   92          1
Length: 6239, dtype: int64

In [34]:
df_07_barra['month'].unique() ## el campo de me tiene que tener hasta 12 calores

array(['11', '2', '8', '3', '7', '5', '1', '12', '6', '9', '4', '10'],
      dtype=object)

In [35]:
df_07_barra['day'].unique() ## el campo de día tiene que tener hasta 31 valores

array(['1', '28', '20', '31', '25', '12', '14', '8', '21', '10', '24',
       '27', '30', '26', '4', '16', '15', '23', '3', '6', '19', '18',
       '17', '13', '22', '9', '29', '11', '7', '2', '5'], dtype=object)

In [36]:
df_07_barra['decade'].unique()

array(['66', '83', '78', '05', '95', '64', '72', '71', '55', '02', '84',
       '94', '96', '99', '74', '14', '20', '13', '18', '15', '11', '17',
       '93', '70', '88', '48', '97', '04', '59', '54', '53', '56', '57',
       '73', '79', '82', '08', '12', '63', '03', '90', '91', '98', '86',
       '75', '00', '92', '65', '67', '62', '47', '87', '76', '89', '09',
       '58', '77', '68', '07', '10', '52', '61', '80', '06', '19', '44',
       '16', '37', '60', '69', '85', '30', '38', '01', '81', '45', '28',
       '24', '35', '40', '26', '21', '29', '51', '42', '39', '41', '27',
       '36', '32', '46', '50', '49', '34', '31', '43', '25', '33', '23',
       '22'], dtype=object)

In [37]:
df_07_barra['date'] = df_07_barra['day'] + '/' + df_07_barra['month'] + '/' + df_07_barra['year'].astype(str)
df_07_barra['date'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\3754407283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_07_barra['date'] = df_07_barra['day'] + '/' + df_07_barra['month'] + '/' + df_07_barra['year'].astype(str)


1/12/1952    148
12/6/2020    145
1/12/1947    134
1/12/1948    110
16/6/1966    103
            ... 
10/9/1990      1
28/7/2006      1
20/5/1968      1
25/7/1974      1
19/4/2004      1
Name: date, Length: 6239, dtype: int64

### largo del campo es 8
como ya hemos dicho, el largo del campo es 4 -> el campo es directamente el año. forzaremos a que el primer día del mes sea el día de pulicación del disco.

In [38]:
df_08 = df[df['release_date_length'] == 8]
df_08['release_date'].value_counts()

11/13/42    384
12/31/48    267
12/31/47    239
12/31/49    182
12/31/43    149
           ... 
12/12/86      1
12/31/98      1
12/15/90      1
10/10/84      1
12/25/00      1
Name: release_date, Length: 1786, dtype: int64

In [39]:
### comprobar cual es el separador de los datos de fecha
print((1 - (df_08.shape[0] - df_08['release_date'].str.contains('-').sum())/df_08.shape[0])*100, 
    (1 - (df_08.shape[0] - df_08['release_date'].str.contains('/').sum())/df_08.shape[0])*100)
##no hay valores raros, podemos continuar con el procedimiento habitual

0.0 100.0


In [40]:
df_08[['month', 'day', 'decade']] = df_08['release_date'].str.split('/', expand=True)
df_08[['month', 'day', 'decade']] .value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\2783975961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_08[['month', 'day', 'decade']] = df_08['release_date'].str.split('/', expand=True)
C:\Users\annas\AppData\Local\Temp\ipykernel_23844\2783975961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_08[['month', 'day', 'decade']] = df_08['release_date'].str.split('/', expand=True)


month  day  decade
11     13   42        384
12     31   48        267
            47        239
            49        182
            43        149
                     ... 
       13   14          1
10     17   74          1
12     13   10          1
11     17   91          1
10     16   05          1
Length: 1786, dtype: int64

In [41]:
df_08['month'].unique() ## el campo de me tiene que tener hasta 12 calores

array(['11', '12', '10'], dtype=object)

In [42]:
df_08['day'].unique() ## el campo de día tiene que tener hasta 31 valores

array(['28', '17', '10', '15', '22', '21', '13', '11', '12', '16', '18',
       '23', '25', '20', '26', '31', '19', '14', '30', '29', '24', '27'],
      dtype=object)

In [43]:
df_08['decade'].unique()

array(['14', '15', '13', '17', '75', '42', '70', '88', '58', '51', '66',
       '08', '78', '05', '98', '71', '03', '93', '81', '83', '63', '80',
       '11', '74', '62', '59', '04', '07', '57', '86', '09', '77', '55',
       '89', '00', '68', '16', '02', '99', '18', '10', '19', '01', '12',
       '97', '31', '82', '85', '64', '69', '96', '92', '73', '90', '95',
       '67', '34', '87', '76', '60', '94', '39', '91', '29', '84', '30',
       '06', '72', '61', '48', '45', '40', '38', '46', '37', '50', '53',
       '43', '47', '52', '65', '41', '49', '54', '44', '36', '32', '56',
       '79', '24', '33', '35', '25', '21', '23'], dtype=object)

In [44]:
df_08['date'] = df_08['day'] + '/' + df_08['month'] + '/' + df_08['year'].astype(str)
df_08['date'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\4173872011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_08['date'] = df_08['day'] + '/' + df_08['month'] + '/' + df_08['year'].astype(str)


13/11/1942    384
31/12/1948    267
31/12/1947    239
31/12/1949    182
31/12/1943    149
             ... 
12/12/1986      1
31/12/1998      1
15/12/1990      1
10/10/1984      1
25/12/2000      1
Name: date, Length: 1786, dtype: int64

### largo del campo es 4
como ya hemos dicho, el largo del campo es 4 -> el campo es directamente el año. forzaremos a que el primer día del mes sea el día de pulicación del disco.

In [45]:
df_04 = df[df['release_date_length'] == 4]
df_04['release_date'].value_counts()

1945    1449
1949    1254
1935    1123
1948    1052
1951     986
        ... 
2014      37
1922      27
2015      12
2016       5
2017       4
Name: release_date, Length: 97, dtype: int64

In [46]:
### comprobar cual es el separador de los datos de fecha -- > en este caso tiene que ser nulo
print((1 - (df_04.shape[0] - df_04['release_date'].str.contains('-').sum())/df_04.shape[0])*100, 
    (1 - (df_04.shape[0] - df_04['release_date'].str.contains('/').sum())/df_04.shape[0])*100)

0.0 0.0


In [47]:
print(df_04.shape)
df_04.sample(5)

(50382, 22)


,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length,month,day
152554,19oDwfNgvYD8OzE8j00Zve,"Verdi : Ernani : Part 4: La maschera ""Elvira!....","['Giuseppe Verdi', 'Fernando Previtali', 'Cate...",101160,1951,1951,0.987,0.327,0.178,0.026900,0.331,-14.580,0.0515,89.860,0.081,1,7,0,0,4,,
121252,0md3dDLRwrsgBQ59CfgmXO,She's Got to Be a Saint,['Ray Price'],228267,1956,1956,0.858,0.414,0.299,0.212000,0.128,-12.293,0.0303,88.516,0.447,1,10,18,0,4,,
23124,2gKIaMUAXrVgjJBpxbPrSM,"Cello Suite No. 3 in C Major, BWV 1009: III. C...","['Johann Sebastian Bach', 'Yo-Yo Ma']",147467,1983,1983,0.935,0.380,0.157,0.802000,0.154,-24.751,0.0483,109.094,0.871,1,7,31,0,4,,
27352,7h6rWJPResRa8xkvnJpxqY,Concierto de Aranjuez: I. Allegro con spirito,"['Daniel Barenboim', 'John Williams', 'English...",357653,1967,1967,0.925,0.506,0.291,0.879000,0.102,-15.577,0.0331,123.639,0.290,1,2,32,0,4,,
33169,1nnxhuI4KGB5EVpqoyrpkQ,"Die Borger, Kapitel 16","['Mary Norton', 'Katharina Thalbach']",273467,1952,1952,0.499,0.615,0.350,0.000001,0.271,-20.947,0.9520,60.094,0.566,1,1,3,0,4,,


en este caso no tenemos valores para `month` o para `day`. podemos suponer que el valor que tomará será la moda de los meses de los datos que ya hemos tratado.

In [48]:
moda = pd.concat([df_06, df_07_barra, df_07_guion, df_08], axis = 0)['month'].mode()[0]
moda

'1'

In [49]:
df_04['month'] = moda
df_04['month'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\1507489564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_04['month'] = moda


1    50382
Name: month, dtype: int64

In [50]:
df_04['date'] = '01/' + df_04['month'].astype(str) + '/' + df_04['year'].astype(str)
df_04['date'].value_counts()

C:\Users\annas\AppData\Local\Temp\ipykernel_23844\2783127687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_04['date'] = '01/' + df_04['month'].astype(str) + '/' + df_04['year'].astype(str)


01/1/1945    1449
01/1/1949    1254
01/1/1935    1123
01/1/1948    1052
01/1/1951     986
             ... 
01/1/2014      37
01/1/1922      27
01/1/2015      12
01/1/2016       5
01/1/2017       4
Name: date, Length: 97, dtype: int64

## concat values

In [51]:
df_fixed = pd.concat([df_04, df_06, df_07_barra, df_07_guion, df_08])
print(df_fixed.shape)
df_fixed.head(1)

(169907, 24)


,id,name,artists,duration_ms,release_date,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode,key,popularity,explicit,release_date_length,month,day,date,decade
6,3mdPsKBeXxC4N6oKKZeBNy,"""8"" Teen",['? & The Mysterians'],166693,1966,1966,0.111,0.657,0.872,0.713,0.0857,-7.279,0.0512,136.512,0.738,1,2,17,0,4,1,,01/1/1966,NaN


In [57]:
df_fixed['date'] = pd.to_datetime(df_fixed['date'], dayfirst=True)
df_fixed['day'] = df_fixed['date'].dt.day
df_fixed['month'] = df_fixed['date'].dt.month
df_fixed['decade'] = df_fixed['date'].dt.year - (df_fixed['date'].dt.year %10)
df_fixed.dtypes

id                             object
name                           object
artists                        object
duration_ms                     int64
release_date                   object
year                            int64
acousticness                  float64
danceability                  float64
energy                        float64
instrumentalness              float64
liveness                      float64
loudness                      float64
speechiness                   float64
tempo                         float64
valence                       float64
mode                            int64
key                             int64
popularity                      int64
explicit                        int64
release_date_length             int64
month                           int64
day                             int64
date                   datetime64[ns]
decade                          int64
dtype: object

In [58]:
df_fixed['decade'].value_counts()

1960    20000
1980    20000
1990    20000
2000    20000
1970    19998
1950    19950
2010    19900
1940    14968
1930     8889
1920     4446
2020     1756
Name: decade, dtype: int64